In [16]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from utils import PkdbModel, codeine_idx, pktype_data, filter_out, convert_unit, unstring
plt.style.use('seaborn-whitegrid')
import numpy as np

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.data.loc[(all_complete.data[("sex","choice")] == "M"),("oral contraceptives","choice")] = "N"
complete_codeine = all_complete.data[codeine_idx(all_complete.data)]

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (20,23,24,25,36,80,82,86,100,102,104,111,115,120,121,122,126,134,135,136,144,153,154,158,161,162,164,165,166,167,168,176,179,180,181,189,196,197,204,209,214,219,224,232,238,239,246,252,258,268) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [19]:
codeine_clearance = pktype_data(complete_codeine,"clearance")

codeine_clearance[("weight","value")] = pd.to_numeric(codeine_clearance[("weight","value")])
codeine_clearance[("weight","mean")] = pd.to_numeric(codeine_clearance[("weight","mean")])
codeine_clearance["value_intervention"] = pd.to_numeric(codeine_clearance["value_intervention"])
codeine_clearance["median"] = pd.to_numeric(codeine_clearance["median"])
codeine_clearance["mean"] = pd.to_numeric(codeine_clearance["mean"])
codeine_clearance["value"] = pd.to_numeric(codeine_clearance["value"])
codeine_clearance["se"] = pd.to_numeric(codeine_clearance["se"])
codeine_clearance["sd"] = pd.to_numeric(codeine_clearance["sd"])
codeine_clearance["cv"] = pd.to_numeric(codeine_clearance["cv"])

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [20]:
codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_clearance[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

codeine_clearance_abs = convert_unit(codeine_clearance,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_clearance[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])

codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs], ignore_index=True)

codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="l/hr",
                             unit_out="l/hr/kg", 
                             factor=1.0/codeine_clearance[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'], 
                             subset=[("weight","value"),"value"])

codeine_clearance_abs_i = convert_unit(codeine_clearance,
                             unit_in="l/hr/kg",
                             unit_out="l/hr", 
                             factor=codeine_clearance[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'],
                             subset=[("weight","value"),"value"])

codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs_i], ignore_index=True)

codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_clearance[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

codeine_clearance_abs = convert_unit(codeine_clearance,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_clearance[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs], ignore_index=True)


codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="l/h",
                             unit_out="l/h/kg", 
                             factor=1.0/codeine_clearance[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])

codeine_clearance_abs = convert_unit(codeine_clearance,
                             unit_in="l/h/kg",
                             unit_out="l/h", 
                             factor=codeine_clearance[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs], ignore_index=True)

In [21]:
codeine_clr = PkdbModel("codeine_clearance",destination="3-inferred")
codeine_clr.data = codeine_clearance
codeine_clr.save()
codeine_clearance.groupby(["unit","unit_intervention"]).count()

study  output_type  pk  raw_pk  \
unit                    unit_intervention                                   
liter / hour / kilogram gram                  17           17  17      17   
                        gram / kilogram        3            3   3       3   
milliliter / hour       gram                  60           60  60      60   

                                           tissue  interventions  \
unit                    unit_intervention                          
liter / hour / kilogram gram                   17             17   
                        gram / kilogram         3              3   
milliliter / hour       gram                   60             60   

                                           substance_name  pktype  calculated  \
unit                    unit_intervention                                       
liter / hour / kilogram gram                           17      17          17   
                        gram / kilogram                 3       3           3   
milliliter / hour       gram                           60      60          60   

                                           individual_pk  ...  \
unit                    unit_intervention                 ...   
liter / hour / kilogram gram                          14  ...   
                        gram / kilogram                0  ...   
milliliter / hour       gram                          28  ...   

                                           (metabolic ratio, count)  \
unit                    unit_intervention                             
liter / hour / kilogram gram                                      0   
                        gram / kilogram                           0   
milliliter / hour       gram                                      0   

                                           (metabolic ratio, max)  \
unit                    unit_intervention                           
liter / hour / kilogram gram                                    0   
                        gram / kilogram                         0   
milliliter / hour       gram                                    0   

                                           (metabolic ratio, mean)  \
unit                    unit_intervention                            
liter / hour / kilogram gram                                     0   
                        gram / kilogram                          0   
milliliter / hour       gram                                     0   

                                           (metabolic ratio, min)  \
unit                    unit_intervention                           
liter / hour / kilogram gram                                    0   
                        gram / kilogram                         0   
milliliter / hour       gram                                    0   

                                           (metabolic ratio, unit)  \
unit                    unit_intervention                            
liter / hour / kilogram gram                                     0   
                        gram / kilogram                          0   
milliliter / hour       gram                                     0   

                                           (fat weight, count)  \
unit                    unit_intervention                        
liter / hour / kilogram gram                                 0   
                        gram / kilogram                      0   
milliliter / hour       gram                                 0   

                                           (fat weight, cv)  \
unit                    unit_intervention                     
liter / hour / kilogram gram                              0   
                        gram / kilogram                   0   
milliliter / hour       gram                              0   

                                           (fat weight, mean)  \
unit                    unit_intervention                       
liter / hour / kilogram gram                             

## Timecourses

In [22]:
codeine_timecourse = pktype_data(complete_codeine,"concentration")
codeine_timecourse = filter_out(codeine_timecourse,"output_type",["outputs"])
codeine_timecourse = filter_out(codeine_timecourse,"unit",["nmol/l"])
codeine_timecourse = filter_out(codeine_timecourse,"unit",["nmol/ml"])

codeine_timecourse.groupby(["unit","unit_intervention"]).count()

study  output_type  pk  raw_pk  tissue  \
unit         unit_intervention                                           
gram / liter gram                  16           16  16      16      16   
             gram / kilogram        2            2   2       2       2   
mole / liter gram                  20           20  20      20      20   
             gram / kilogram        1            1   1       1       1   

                                interventions  substance_name  pktype  \
unit         unit_intervention                                          
gram / liter gram                          16              16      16   
             gram / kilogram                2               2       2   
mole / liter gram                          20              20      20   
             gram / kilogram                1               1       1   

                                calculated  individual_pk  ...  \
unit         unit_intervention                             ...   
gram / liter gram                        0              2  ...   
             gram / kilogram             0              0  ...   
mole / liter gram                        0             11  ...   
             gram / kilogram             0              0  ...   

                                (metabolic ratio, count)  \
unit         unit_intervention                             
gram / liter gram                                      0   
             gram / kilogram                           0   
mole / liter gram                                      0   
             gram / kilogram                           0   

                                (metabolic ratio, max)  \
unit         unit_intervention                           
gram / liter gram                                    0   
             gram / kilogram                         0   
mole / liter gram                                    0   
             gram / kilogram                         0   

                                (metabolic ratio, mean)  \
unit         unit_intervention                            
gram / liter gram                                     0   
             gram / kilogram                          0   
mole / liter gram                                     0   
             gram / kilogram                          0   

                                (metabolic ratio, min)  \
unit         unit_intervention                           
gram / liter gram                                    0   
             gram / kilogram                         0   
mole / liter gram                                    0   
             gram / kilogram                         0   

                                (metabolic ratio, unit)  (fat weight, count)  \
unit         unit_intervention                                                 
gram / liter gram                                     0                    0   
             gram / kilogram                          0                    0   
mole / liter gram                                     0                    0   
             gram / kilogram                          0                    0   

                                (fat weight, cv)  (fat weight, mean)  \
unit         unit_intervention                                         
gram / liter gram                              0                   0   
             gram / kilogram                   0                   0   
mole / liter gram                              0                   0   
             gram / kilogram                   0                   0   

                                (fat weight, se)  (fat weight, unit)  
unit         unit_intervention                                        
gram / liter gram                              0                   0  
             gram / kilogram                   0                   0  
mole / liter gram                              0                   0  
             gram / kilogram                   0                   0

In [23]:
codeine_tc = PkdbModel("codeine_timecourse",destination="3-inferred")
codeine_tc.data = codeine_timecourse
codeine_tc.to_array()
codeine_timecourse = codeine_tc.data

In [24]:
codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_timecourse[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

codeine_timecourse_abs = convert_unit(codeine_timecourse,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_timecourse[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])



codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel,codeine_timecourse_abs], ignore_index=True)


codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="µg/ml",
                             unit_out="µg/ml/kg", 
                             factor=1.0/codeine_timecourse[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'], 
                             subset=[("weight","value"),"value"])

codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel], ignore_index=True)

codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_timecourse[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

codeine_timecourse_abs = convert_unit(codeine_timecourse,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_timecourse[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel,codeine_timecourse_abs], ignore_index=True)


codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="µg/ml",
                             unit_out="µg/ml/kg", 
                             factor=1.0/codeine_timecourse[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel], ignore_index=True)

In [25]:
codeine_timecourse.groupby(["unit","unit_intervention","time_unit"]).count()

study  output_type  pk  raw_pk  \
unit         unit_intervention time_unit                                   
gram / liter gram              hr            16           16  16      16   
             gram / kilogram   hr             2            2   2       2   
mole / liter gram              hr            20           20  20      20   
             gram / kilogram   hr             1            1   1       1   

                                          tissue  interventions  \
unit         unit_intervention time_unit                          
gram / liter gram              hr             16             16   
             gram / kilogram   hr              2              2   
mole / liter gram              hr             20             20   
             gram / kilogram   hr              1              1   

                                          substance_name  pktype  calculated  \
unit         unit_intervention time_unit                                       
gram / liter gram              hr                     16      16           0   
             gram / kilogram   hr                      2       2           0   
mole / liter gram              hr                     20      20           0   
             gram / kilogram   hr                      1       1           0   

                                          individual_pk  ...  \
unit         unit_intervention time_unit                 ...   
gram / liter gram              hr                     2  ...   
             gram / kilogram   hr                     0  ...   
mole / liter gram              hr                    11  ...   
             gram / kilogram   hr                     0  ...   

                                          (metabolic ratio, count)  \
unit         unit_intervention time_unit                             
gram / liter gram              hr                                0   
             gram / kilogram   hr                                0   
mole / liter gram              hr                                0   
             gram / kilogram   hr                                0   

                                          (metabolic ratio, max)  \
unit         unit_intervention time_unit                           
gram / liter gram              hr                              0   
             gram / kilogram   hr                              0   
mole / liter gram              hr                              0   
             gram / kilogram   hr                              0   

                                          (metabolic ratio, mean)  \
unit         unit_intervention time_unit                            
gram / liter gram              hr                               0   
             gram / kilogram   hr                               0   
mole / liter gram              hr                               0   
             gram / kilogram   hr                               0   

                                          (metabolic ratio, min)  \
unit         unit_intervention time_unit                           
gram / liter gram              hr                              0   
             gram / kilogram   hr                              0   
mole / liter gram              hr                              0   
             gram / kilogram   hr                              0   

                                          (metabolic ratio, unit)  \
unit         unit_intervention time_unit                            
gram / liter gram              hr                               0   
             gram / kilogram   hr                               0   
mole / liter gram              hr                               0   
             gram / kilogram   hr                               0   

                                          (fat weight, count)  \
unit         unit_intervention time_unit                        
gram / liter gram              hr                           0   
             gram / kilogram   hr                   

In [26]:
codeine_tc = PkdbModel("codeine_timecourse",destination="3-inferred")
codeine_tc.data = codeine_timecourse
codeine_tc.save()